In [ ]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
from astropy.io import fits
from scipy import optimize
import poppy
import gpipsfs
import time

Construct the aperture 

In [ ]:
outD = 7.77010            # primary diameter (m)
inD = 1.024               # inner M2 diameter (m)
n = 48                    # number sample points across the screen (Not the number of subapertures)
nacross = 43              # number of subapertures across the aperture
pscale = outD/(nacross)   # pixel size (m) of samples in pupil plane

x = np.linspace(-(n)/2,(n)/2,n)*pscale 
y = np.linspace(-(n)/2,(n)/2,n)*pscale
mg = np.meshgrid(x,y)
ar = np.sqrt(np.sum((m**2 for m in mg)))
ap_outer = (ar <= (7.3)/2) # mask is slightly oversized because GPI does not correct the boundaries well
ap_inner = (ar <= 1.5/2)   
ap = (ap_outer ^ ap_inner).astype(np.float)

ap_nan = np.copy(ap.astype(np.float))  
ap_nan[np.where(ap==0)] = np.nan

In [ ]:
# Laptop path for two telemetry movies
dstr = time.strftime('%Y%m%d')
rootdir = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/'
file1 =  rootdir+'aotelem/aored_When_2016.2.27_0.40.14_poldm_phase.fits'
file2 =  rootdir+'aotelem/aored_When_2016.2.29_22.52.44_poldm_phase.fits'
savepath = rootdir+'Results/psds_general/'
df_avg_sp_psd = pd.read_csv(rootdir+'datatables/campaign_all_avg_sp_psd_20190318.txt',index_col=0)
df_avg_t_psd = pd.read_csv(rootdir+'datatables/campaign_all_avg_t_psd_20190318.txt',index_col=0)

## Spatial PSD grid

In [ ]:
kx = fft.fftshift(fft.fftfreq(n,pscale))
ky = fft.fftshift(fft.fftfreq(n,pscale))
mg = np.meshgrid(kx,ky)
kr = np.sqrt(np.sum((m**2 for m in mg)))

def radialProfile(image, center=None):
    """
    Calculate the avearge radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    ## Calculate the indices from the image
    y,x = np.indices((image.shape)) # first determine radii of all pixels
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
     
    r = np.hypot(x - center[0], y - center[1]).astype(np.int) 
    
    n = np.bincount(r.ravel())
    sy = np.bincount(r.ravel(), image.ravel())
    mean = sy/n
    
    return mean

k = radialProfile(kr)

In [ ]:
%matplotlib notebook

n = 1
fig = plt.figure(figsize=[10,7])
fig.subplots_adjust(hspace=0.4, wspace=0.4)

def power_law_fit(k,Y,low_b,up_b):

    par = np.polyfit(np.log10(k[(k>low_b) & (k<up_b)]), np.log10(Y[(k>low_b) & (k<up_b)]), 1)
    exp = par[0]
    amp = par[1]
    print(exp,amp)
    return exp, amp   

def power_law(k,amp,exp):
    return 10**amp*k**exp

for i in df_avg_sp_psd.columns[4:104:25]:
    psd = df_avg_sp_psd[i]
    exp,amp = power_law_fit(k,psd,.3,1)
    kfit = k[(k>.3) & (k<1.0)]
    y = power_law(kfit,amp,exp)
    
    ax = fig.add_subplot(2, 2, n)
    #ax.loglog(kfit,y,label = str(np.round(exp,2)),lw = 3,color='r')
    ax.loglog(k,psd,label='exp = '+str(np.round(exp,2)))
    ax.legend()
    ax.axvline(0.3,ls = '--',color = 'silver')
    ax.axvline(1,ls = '--',color = 'silver')
    ax.set_title(i)
    n = n + 1 
    
plt.savefig(savepath+'psd_grid_4_'+dstr+'.png')



## Comparison of temporal modes

In [ ]:
def process_phase(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')
    avg_phase = np.nanmean(phase*ap_nan,axis=0)  # used to find average zernikes 

    # remove zernikes form cube
    z_basis = poppy.zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = poppy.zernike.opd_expand_nonorthonormal(avg_phase, aperture=ap, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)

    c_phase = (phase - thin_lens[None,:,:])*ap_nan
    c_phase[np.isnan(c_phase)]=0.
    
    return c_phase

def convert_to_modal_basis(phase, ap):
    
    timesteps, phx, phy = phase.shape 
    phFT = np.zeros((timesteps,phx,phy), dtype=complex)
    norm = 1.0/np.sqrt(ap.sum())
    for t in np.arange(timesteps):
        phFT[t,:,:] = np.fft.fftshift(fft.fft2(phase[t,:,:]*ap))*norm
    print('Done with FT')

    return phFT

def temp_power_spec(Y):
    
    n = len(Y)
    P = np.fft.fft(Y)
    norm = 1.0/n
    P = P * norm
    P2 = np.square(np.abs(P))
    
    return P2

In [ ]:
%matplotlib notebook

y_2D = process_phase(file1)
y_1D = np.mean(y_2D,axis=(1,2))

# First do the average of actuator positions 
kt = fft.fftfreq(len(y_1D),.001)
keep = kt > 0.0 
kt_keep = kt[keep]

fig = plt.figure(figsize=[12,4])
#fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax1 = fig.add_subplot(1,3,1)
t_psd = temp_power_spec(y_1D)
#t_exp,t_amp = power_law_fit(k,t_psd,1,40)
ax1.loglog(kt_keep,t_psd[keep])
ax1.set_title('AVG actuator position time series')

ax2 = fig.add_subplot(1,3,2)
yFT_2D = convert_to_modal_basis(y_2D,ap)
mode = yFT_2D[:,4,4]
m_psd = temp_power_spec(mode)
ax2.loglog(kt_keep,m_psd[keep])
ax2.set_title('MODE [4,4]')

ax3 = fig.add_subplot(1,3,3)
f, psd = welch(y_1D,
               fs=.001,  # sample rate
               window='blackman',   # apply a Hanning window before taking the DFT
               nperseg=1024,        # compute periodograms of 256-long segments of x
               detrend='constant') # detrend x by subtracting the mean
keep = f>0.0
f_keep = f[keep]
ax3.loglog(f[keep],psd[keep])
ax3.set_title('actuators with overlapping segments (1024 ms)')
t_exp,t_amp = power_law_fit(f_keep,psd[keep],100,400)

fig.tight_layout()
#plt.savefig(savepath+'t_psd_3_dif_ways_2'+dstr+'.png')

